In [340]:
import time
import datetime
import json
import os
import random
import numpy as np
import webbrowser
from PIL import Image
import matplotlib.pyplot as plt
from rich.progress import track
import subprocess
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder, DatasetFolder
from torch.utils.data import Dataset
from torchinfo import summary
from sklearn.preprocessing import MinMaxScaler

In [341]:
transforms = transforms.Compose([
    transforms.Grayscale(),
    transforms.PILToTensor()])


In [342]:
train_dataset = ImageFolder(root="pics_with_coords", transform=transforms)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [343]:
class LinNet(nn.Module):
    def __init__(self):
        super(LinNet, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(2048, 512),
            nn.SiLU(),
            nn.Linear(512, 2)
        )

    def forward(self, x):
        return self.model(x)

In [344]:
data_iter = iter(train_dataloader)
data = next(data_iter)

x, _ = data
x = x.float()


In [345]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
filenames = [train_dataloader.dataset.imgs[i][0].split('/')[-1] for i in range(len(train_dataset))]
len(filenames)

1996

In [346]:
lat_scaler = MinMaxScaler(feature_range=(-1, 1))
lon_scaler = MinMaxScaler(feature_range=(-1, 1))

img_num = 1 

lat = train_dataloader.dataset.imgs[img_num][0].strip(".png").split("_")[-2]
lon = train_dataloader.dataset.imgs[img_num][0].strip(".png").split("_")[-1]

filename = train_dataloader.dataset.imgs[img_num][0].split('/')[-1]

lat_scaler.fit(np.array([[-90], [90]]))
lon_scaler.fit(np.array([[-180], [180]]))

lat = np.array([[float(lat)]])
lon = np.array([[float(lon)]])
scaled_lat = lat_scaler.transform(lat)
scaled_lon = lon_scaler.transform(lon)
inv_scaled_lat = lat_scaler.inverse_transform(scaled_lat)
inv_scaled_lon = lon_scaler.inverse_transform(scaled_lon)

print(filename)
print(lat, lon)
print(scaled_lat, scaled_lon)
print(inv_scaled_lat, inv_scaled_lon)

epic_RGB_20150803015633_9.089355_159.250488.png
[[9.089355]] [[159.250488]]
[[0.10099283]] [[0.88472493]]
[[9.089355]] [[159.250488]]


In [359]:
for num, image in enumerate(filenames):
    '''
    1. Extract lat and lon
    2. Scale lat and lon to (-1, 1)
    3. Make Tensor for criterion
    '''
    lat = train_dataloader.dataset.imgs[num][0].strip(".png").split("_")[-2]
    lon = train_dataloader.dataset.imgs[num][0].strip(".png").split("_")[-1]
    lat = np.array([[float(lat)]])
    lon = np.array([[float(lon)]])
    scaled_lat = lat_scaler.transform(lat)
    scaled_lon = lon_scaler.transform(lon)

    print(lat, scaled_lat)
    print(lon, scaled_lon)

    target = torch.Tensor([scaled_lon[0], scaled_lon[0]])
    print(target)

    print(image)
    break


[[9.09668]] [[0.10107422]]
[[-173.71582]] [[-0.96508789]]
tensor([[-0.9651],
        [-0.9651]])
epic_RGB_20150803000829_9.09668_-173.71582.png
